In [15]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn.svm import SVR

In [16]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/UrbanComputing/Project')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Read Data

In [17]:
datapath_edge = "Dataset/London_edge_data_500m.csv"
datapath_node = "Dataset/London_node_data_500m.csv"

In [ ]:
df_edge = pd.read_csv(datapath_edge)
df_node = pd.read_csv(datapath_node)

In [ ]:
df_edge.drop('Unnamed: 0',axis=1,inplace=True)
df_node.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df_edge.head(2)

In [ ]:
df_node.head(2)

# Preprocessing Data
Using the column total_loc_flow from edge dataset and adding ot to the nodes dataset

In [ ]:
df_edge['total_loc_flow']

In [ ]:
total_loc_flow_list = []
for i in range(len(df_node)):
  total_loc_flow_list.append(list(df_edge.loc[df_edge['location_1']==df_node.loc[i,'nodeID'],'total_loc_flow'])[0])
df_node['total_loc_flow'] = total_loc_flow_list

In [ ]:
# df_edge.loc[df_edge['location_1']==df_node.loc[0,'nodeID'],'total_loc_flow'][0]

In [ ]:
df_node

In [ ]:
print(' ,'.join(list(df_node.columns)))

In [ ]:
df_node.shape

In [ ]:
df_node[['in_total','out_total','total_loc_flow']]

In [ ]:
# df_node[['in_total','out_total', 'total_loc_flow']]

In [ ]:
df_node1 = df_node.copy()
df_node = df_node1

In [ ]:
# Removing 0 flows
# df_node = df_node.loc[df_node['total_loc_flow']!=0,:]
# df_node.loc[df_node['total_loc_flow']==0,:]

In [ ]:
# # Set 1
# df_node = df_node.sample(frac=1)
# X = df_node[['airbnb_price', 'universities', 'tourism', 'theatres',
#         'shops', 'shopping_malls', 'restaurants', 'residential', 'pubs_cafes',
#         'post', 'parking', 'offices', 'museums', 'medical', 'schools',
#         'industrial', 'government', 'fuels', 'fast_foods', 'commercial',
#         'cinemas', 'bars_cafes', 'banks', 'atms', 'arts', 'airport','street_density', 'gyration_rad', 'highways', 'metro_flow','in_total_spatial_lag', 'out_total_spatial_lag',
#         'gyration_radius_spatial_lag', 'in_total',
#         'out_total']]
# Y = df_node[['total_loc_flow']]

In [ ]:
# Y

In [ ]:
# # Set 2
X = df_node[['airbnb_price', 'universities', 'tourism', 'theatres',
        'shops', 'shopping_malls', 'restaurants', 'residential', 'pubs_cafes',
        'post', 'parking', 'offices', 'museums', 'medical', 'schools',
        'industrial', 'government', 'fuels', 'fast_foods', 'commercial',
        'cinemas', 'bars_cafes', 'banks', 'atms', 'arts', 'airport','street_density', 'gyration_rad', 'highways', 'metro_flow','in_total_spatial_lag', 'out_total_spatial_lag','out_total']]
Y = df_node[['in_total']]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=42)

# Apply ML Models

In [ ]:
# compare_df = pd.DataFrame( columns =  ['Classifier','MAE','MSE','R2','MAPE'])
classifier = []
MAE=[]
MSE = []
R2=[]
MAPE=[]

## Naive Bayesian 

In [ ]:
gnbmodel = GaussianNB()
gnbmodel.fit(x_train,y_train.values.ravel())
y_pred_dt = gnbmodel.predict(x_test)
(precision, recall, fscore, support) = precision_recall_fscore_support(y_test, y_pred_dt, average='macro')
print('Precision: {}\tRecall: {}\tfscore:{}'.format(precision, recall, fscore))
# compare_df.loc[len(compare_df)]=['Naive-Bayes',precision, recall, fscore]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_pred_dt, y_test, edgecolors=(0, 0, 1))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title("Naive Bayesian")
plt.savefig("NaiveBayes", format="PNG")
plt.show()

In [ ]:
# model evaluation for testing set

mae = metrics.mean_absolute_error(y_test, y_pred_dt)
mse = metrics.mean_squared_error(y_test, y_pred_dt)
r2 = metrics.r2_score(y_test, y_pred_dt)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred_dt)

classifier.append('Naive Bayes')
MAE.append(mae)
MSE.append(mse)
R2.append(r2)
MAPE.append(mape)
	

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('mape score is {}'.format(mape))

In [ ]:
# compare_df

## Logistic Regression

In [ ]:
logisticRegr = LogisticRegression()
logisticRegr.fit(x_train,y_train.values.ravel())
y_pred_dt = logisticRegr.predict(x_test)
(precision, recall, fscore, support) = precision_recall_fscore_support(y_test, y_pred_dt, average='macro')
print('Precision: {}\tRecall: {}\tfscore:{}'.format(precision, recall, fscore))
# compare_df.loc[len(compare_df)]=['Logistic Regression',precision, recall, fscore]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_pred_dt, y_test, edgecolors=(0, 0, 1))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title("Logistic Regression")
# plt.savefig("LogReg", format="PNG")
plt.show()

In [ ]:
# model evaluation for testing set

mae = metrics.mean_absolute_error(y_test, y_pred_dt)
mse = metrics.mean_squared_error(y_test, y_pred_dt)
r2 = metrics.r2_score(y_test, y_pred_dt)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred_dt)

classifier.append('Logistic Regression')
MAE.append(mae)
MSE.append(mse)
R2.append(r2)
MAPE.append(mape)
	

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('mape score is {}'.format(mape))

## Decission Tree Regressor

In [ ]:
dtmodel = DecisionTreeRegressor(random_state=0)
print('Decision Tree')
dtmodel.fit(x_train,y_train.values.ravel())
y_pred_dt = dtmodel.predict(x_test)
# (precision, recall, fscore, support) = precision_recall_fscore_support(y_test, y_pred_dt, average='macro')
# print('Precision: {}\tRecall: {}\tfscore:{}'.format(precision, recall, fscore))
# compare_df.loc[len(compare_df)]=['Decision Tree',precision, recall, fscore]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_pred_dt, y_test, edgecolors=(0, 0, 1))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title("Decission Tree Regressor")
# plt.savefig("DTR", format="PNG")
plt.show()

In [ ]:
# model evaluation for testing set

mae = metrics.mean_absolute_error(y_test, y_pred_dt)
mse = metrics.mean_squared_error(y_test, y_pred_dt)
r2 = metrics.r2_score(y_test, y_pred_dt)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred_dt)

classifier.append('Decision Tree Regressor')
MAE.append(mae)
MSE.append(mse)
R2.append(r2)
MAPE.append(mape)
	

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('mape score is {}'.format(mape))

# SVM

In [ ]:
regr  = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2,kernel='linear'))
regr .fit(x_train,y_train.values.ravel())
y_pred_dt = regr.predict(x_test)
# (precision, recall, fscore, support) = precision_recall_fscore_support(y_test, y_pred_dt, average='macro')
# print('Precision: {}\tRecall: {}\tfscore:{}'.format(precision, recall, fscore))
# compare_df.loc[len(compare_df)]=['SVM',precision, recall, fscore]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_pred_dt, y_test, edgecolors=(0, 0, 1))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title("SVM")
# plt.savefig("SVM", format="PNG")
plt.show()

In [ ]:
# model evaluation for testing set

mae = metrics.mean_absolute_error(y_test, y_pred_dt)
mse = metrics.mean_squared_error(y_test, y_pred_dt)
r2 = metrics.r2_score(y_test, y_pred_dt)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred_dt)

classifier.append('SVM')
MAE.append(mae)
MSE.append(mse)
R2.append(r2)
MAPE.append(mape)
	

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('mape score is {}'.format(mape))

In [ ]:
# compare_df

## Random Forrest Regressor

In [ ]:
clf = RandomForestRegressor(max_depth=2, random_state=0)
clf.fit(x_train,y_train.values.ravel())
y_pred_dt = clf.predict(x_test)
# (precision, recall, fscore, support) = precision_recall_fscore_support(y_test, y_pred_dt, average='macro')
# print('Precision: {}\tRecall: {}\tfscore:{}'.format(precision, recall, fscore))
# compare_df.loc[len(compare_df)]=['Random Forest',precision, recall, fscore]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_pred_dt, y_test, edgecolors=(0, 0, 1))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=3)
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title("Random Forrest Regressor")
# plt.savefig("RandomForrest", format="PNG")
plt.show()

In [ ]:
# model evaluation for testing set

mae = metrics.mean_absolute_error(y_test, y_pred_dt)
mse = metrics.mean_squared_error(y_test, y_pred_dt)
r2 = metrics.r2_score(y_test, y_pred_dt)
mape = metrics.mean_absolute_percentage_error(y_test, y_pred_dt)

classifier.append('Random Forrest Regressor')
MAE.append(mae)
MSE.append(mse)
R2.append(r2)
MAPE.append(mape)
	

print("The model performance for testing set")
print("--------------------------------------")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 score is {}'.format(r2))
print('mape score is {}'.format(mape))

In [ ]:
# compare_df

In [ ]:
df = pd.DataFrame({'Classifier':classifier,
                  'MAE':MAE,
                   'MSE':MSE,
                   'R2':R2,
                   'MAPE':MAPE})

In [ ]:
MAPE

In [ ]:
df.iloc[:,1:]=df.iloc[:,1:].apply(lambda x : round(x,2) )

In [ ]:
print(df)

In [ ]:
print(df)

In [ ]:
0 	0 	Naive Bayes 	141.10 	125764.04 	0.86 	2.649176e+15